Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import math
import random
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [16]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [17]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## First Greedy Algorithm

In [18]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))
logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

DEBUG:root:step: Ancona -> Rimini (90.60km)
DEBUG:root:step: Rimini -> Forlì (46.72km)
DEBUG:root:step: Forlì -> Ravenna (26.46km)
DEBUG:root:step: Ravenna -> Ferrara (66.67km)
DEBUG:root:step: Ferrara -> Bologna (43.43km)
DEBUG:root:step: Bologna -> Modena (37.29km)
DEBUG:root:step: Modena -> Reggio nell'Emilia (23.94km)
DEBUG:root:step: Reggio nell'Emilia -> Parma (26.94km)
DEBUG:root:step: Parma -> Piacenza (57.65km)
DEBUG:root:step: Piacenza -> Milan (60.65km)
DEBUG:root:step: Milan -> Monza (14.51km)
DEBUG:root:step: Monza -> Bergamo (33.92km)
DEBUG:root:step: Bergamo -> Brescia (46.02km)
DEBUG:root:step: Brescia -> Verona (61.42km)
DEBUG:root:step: Verona -> Vicenza (44.70km)
DEBUG:root:step: Vicenza -> Padua (30.13km)
DEBUG:root:step: Padua -> Venice (36.07km)
DEBUG:root:step: Venice -> Trieste (115.09km)
DEBUG:root:step: Trieste -> Bolzano (209.68km)
DEBUG:root:step: Bolzano -> Trento (49.94km)
DEBUG:root:step: Trento -> Novara (206.69km)
DEBUG:root:step: Novara -> Turin (84.46

In [ ]:


# Define the 2-opt swap function
def two_opt_swap(tour, i, j):
    """Perform a 2-opt swap by reversing the order of cities between indices i and j."""
    new_tour = tour[:i] + tour[i:j+1][::-1] + tour[j+1:]
    return new_tour

# Define scalable Simulated Annealing function
def scalable_simulated_annealing(initial_solution, dist_matrix, num_cities):
    # Set parameters based on the size of the dataset
    initial_temp = max(1000, num_cities * 10)            # Higher starting temperature for larger datasets
    min_temp = 1                                         # Constant minimum temperature
    cooling_rate = 0.995 if num_cities < 1000 else 0.999 # Slower cooling rate for larger instances
    max_iter_per_temp = max(100, num_cities // 10)       # Limit iterations per temperature level

    current_solution = initial_solution
    current_cost = tsp_cost(current_solution)
    best_solution = current_solution
    best_cost = current_cost
    temperature = initial_temp

    iteration = 0
    while temperature > min_temp:
        # Perform a fixed number of iterations per temperature level to scale to larger datasets
        for _ in range(max_iter_per_temp):
            # Generate a neighbor by performing a 2-opt swap
            i, j = sorted(random.sample(range(1, len(current_solution) - 1), 2))
            new_solution = two_opt_swap(current_solution, i, j)
            new_cost = tsp_cost(new_solution)
            
            # Calculate the cost difference
            delta_cost = new_cost - current_cost

            # Decide whether to accept the new solution
            if delta_cost < 0 or random.random() < math.exp(-delta_cost / temperature):
                current_solution = new_solution
                current_cost = new_cost

                # Update the best solution if the new solution is better
                if current_cost < best_cost:
                    best_solution = current_solution
                    best_cost = current_cost

        # Cool down the temperature
        temperature *= cooling_rate
        iteration += 1

        # Logging progress every few iterations
        if iteration % 100 == 0 or temperature <= min_temp:
            print(f"Iteration {iteration}: Current best cost = {best_cost:.2f}, Temperature = {temperature:.2f}")

    return best_solution, best_cost

# Example usage with initial greedy solution `tsp` and distance matrix `DIST_MATRIX`
num_cities = len(DIST_MATRIX)  # Number of cities in the dataset

# Run Simulated Annealing with dynamic scaling
refined_solution, refined_cost = scalable_simulated_annealing(tsp, DIST_MATRIX, num_cities)

# Display the results
print("Initial Greedy Solution Cost:", tsp_cost(tsp))
print("Refined Solution from Scalable Simulated Annealing:", refined_solution)
print("Refined Cost from Scalable Simulated Annealing:", refined_cost)
